In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import parselmouth
from parselmouth.praat import call
import glob

parkinson = pd.read_csv("DataSet/dataset.csv")
parkinson.head()

,Parkinson,Jitter_rel,Jitter_abs,Jitter_RAP,Jitter_PPQ,Shim_loc,Shim_dB,Shim_APQ3,Shim_APQ5,Shi_APQ11,hnr05,hnr15,hnr25
0,1.0,0.019273,0.000149,0.007988,0.008823,0.093380,0.866658,0.035126,0.048810,0.085224,5.031867,5.616350,6.553027
1,1.0,0.031410,0.000281,0.013289,0.013440,0.112340,1.044282,0.041351,0.059329,0.106660,4.302138,4.372703,8.602448
2,1.0,0.025405,0.000205,0.011561,0.012323,0.095643,0.946494,0.035971,0.049964,0.094885,3.931486,4.001019,7.723529
3,1.0,0.022578,0.000167,0.007994,0.010027,0.082393,0.836335,0.027471,0.043146,0.081937,1.985262,4.034309,4.729302
4,1.0,0.031441,0.000164,0.015739,0.016306,0.149788,1.300907,0.055637,0.077863,0.102166,0.377313,2.473989,NaN


In [2]:
parkinson['hnr25'].fillna((parkinson['hnr25'].mean()), inplace=True)
parkinson['hnr15'].fillna((parkinson['hnr15'].mean()), inplace=True)
parkinson.head()

,Parkinson,Jitter_rel,Jitter_abs,Jitter_RAP,Jitter_PPQ,Shim_loc,Shim_dB,Shim_APQ3,Shim_APQ5,Shi_APQ11,hnr05,hnr15,hnr25
0,1.0,0.019273,0.000149,0.007988,0.008823,0.093380,0.866658,0.035126,0.048810,0.085224,5.031867,5.616350,6.553027
1,1.0,0.031410,0.000281,0.013289,0.013440,0.112340,1.044282,0.041351,0.059329,0.106660,4.302138,4.372703,8.602448
2,1.0,0.025405,0.000205,0.011561,0.012323,0.095643,0.946494,0.035971,0.049964,0.094885,3.931486,4.001019,7.723529
3,1.0,0.022578,0.000167,0.007994,0.010027,0.082393,0.836335,0.027471,0.043146,0.081937,1.985262,4.034309,4.729302
4,1.0,0.031441,0.000164,0.015739,0.016306,0.149788,1.300907,0.055637,0.077863,0.102166,0.377313,2.473989,8.594723


In [3]:
predictors=["Jitter_rel","Jitter_abs","Jitter_RAP","Jitter_PPQ","Shim_loc","Shim_dB","Shim_APQ3","Shim_APQ5","Shi_APQ11","hnr05","hnr15", "hnr25"]

In [4]:
for col in predictors: # Loop through all columns in predictors
    if parkinson[col].dtype == 'object':  # check if column's type is object (text)
        parkinson[col] = pd.Categorical(parkinson[col]).codes  # convert text to numerical

parkinson

,Parkinson,Jitter_rel,Jitter_abs,Jitter_RAP,Jitter_PPQ,Shim_loc,Shim_dB,Shim_APQ3,Shim_APQ5,Shi_APQ11,hnr05,hnr15,hnr25
0,1.0,0.019273,0.000149,0.007988,0.008823,0.093380,0.866658,0.035126,0.048810,0.085224,5.031867,5.616350,6.553027
1,1.0,0.031410,0.000281,0.013289,0.013440,0.112340,1.044282,0.041351,0.059329,0.106660,4.302138,4.372703,8.602448
2,1.0,0.025405,0.000205,0.011561,0.012323,0.095643,0.946494,0.035971,0.049964,0.094885,3.931486,4.001019,7.723529
3,1.0,0.022578,0.000167,0.007994,0.010027,0.082393,0.836335,0.027471,0.043146,0.081937,1.985262,4.034309,4.729302
4,1.0,0.031441,0.000164,0.015739,0.016306,0.149788,1.300907,0.055637,0.077863,0.102166,0.377313,2.473989,8.594723
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0.0,0.024510,0.000132,0.010953,0.010540,0.079994,0.771243,0.030688,0.039900,0.066022,1.839533,5.360876,7.631712
69,0.0,0.022260,0.000111,0.010136,0.010165,0.088066,0.896240,0.032909,0.045445,0.081866,1.518921,3.764296,5.547934
70,0.0,0.017001,0.000079,0.008471,0.008114,0.061106,0.633178,0.023786,0.029054,0.047723,0.731566,1.767502,6.397744
71,0.0,0.020385,0.000106,0.009641,0.009898,0.095524,0.908515,0.041091,0.052293,0.081502,7.693267,8.640795,11.301341


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(parkinson[predictors], parkinson['Parkinson'], test_size=0.25, random_state=1)

print ("train shape", X_train.shape, y_train.shape)
print ("test shape", X_test.shape, y_test.shape)

train shape (54, 12) (54,)
test shape (19, 12) (19,)


In [6]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
print ('train accuracy =', train_score)
print ('test accuracy =', test_score)


train accuracy = 0.6666666666666666
test accuracy = 0.631578947368421


In [7]:
from sklearn.model_selection import cross_val_score
clf = LogisticRegression()
scores = cross_val_score(clf, parkinson[predictors], parkinson["Parkinson"], scoring='accuracy', cv=5)

print(scores)
print(scores.mean())



[0.46666667 0.73333333 0.86666667 0.64285714 0.64285714]
0.6704761904761904


In [8]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
# This is the function to measure voice pitch
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)#create a praat pitch object
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    harmonicity05 = call(sound, "To Harmonicity (cc)", 0.01, 500, 0.1, 1.0)
    hnr05 = call(harmonicity05, "Get mean", 0, 0)
    harmonicity15 = call(sound, "To Harmonicity (cc)", 0.01, 1500, 0.1, 1.0)
    hnr15 = call(harmonicity15, "Get mean", 0, 0)
    harmonicity25 = call(sound, "To Harmonicity (cc)", 0.01, 2500, 0.1, 1.0)
    hnr25 = call(harmonicity25, "Get mean", 0, 0)
    harmonicity35 = call(sound, "To Harmonicity (cc)", 0.01, 3500, 0.1, 1.0)
    hnr35 = call(harmonicity35, "Get mean", 0, 0)
    harmonicity38 = call(sound, "To Harmonicity (cc)", 0.01, 3800, 0.1, 1.0)
    hnr38 = call(harmonicity38, "Get mean", 0, 0)
    return localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, hnr05, hnr15 ,hnr25 ,hnr35 ,hnr38


In [26]:
# create lists to put the results
file_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
localShimmer_list =  []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list =  []
hnr05_list = []
hnr15_list = []
hnr25_list = []
hnr35_list = []
hnr38_list = []
parkinson_list = []


sound = parselmouth.Sound("audio/parkin.wav")
(localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, hnr05, hnr15 ,hnr25 ,hnr35 ,hnr38) = measurePitch(sound, 75, 1000, "Hertz")
localJitter_list.append(localJitter) # make a mean F0 list
localabsoluteJitter_list.append(localabsoluteJitter) # make a sd F0 list
rapJitter_list.append(rapJitter)
ppq5Jitter_list.append(ppq5Jitter)
localShimmer_list.append(localShimmer)
localdbShimmer_list.append(localdbShimmer)
apq3Shimmer_list.append(apq3Shimmer)
aqpq5Shimmer_list.append(aqpq5Shimmer)
apq11Shimmer_list.append(apq11Shimmer)
hnr05_list.append(hnr05)
hnr15_list.append(hnr15)
hnr25_list.append(hnr25)
hnr35_list.append(hnr35)
hnr38_list.append(hnr38)

apred = pd.DataFrame(np.column_stack([localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, hnr05_list, hnr15_list, hnr25_list]),
                               columns=["Jitter_rel","Jitter_abs","Jitter_RAP","Jitter_PPQ","Shim_loc","Shim_dB","Shim_APQ3","Shim_APQ5","Shi_APQ11", "hnr05", "hnr15", "hnr25"])  #add these lists to pandas in the right order


In [25]:
clf.predict(apred)


array([1.])

In [36]:
# create lists to put the results
file_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
localShimmer_list =  []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list =  []
hnr05_list = []
hnr15_list = []
hnr25_list = []
hnr35_list = []
hnr38_list = []
parkinson_list = []


sound = parselmouth.Sound("audio/ok.wav")
(localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, hnr05, hnr15 ,hnr25 ,hnr35 ,hnr38) = measurePitch(sound, 75, 1000, "Hertz")
localJitter_list.append(localJitter) # make a mean F0 list
localabsoluteJitter_list.append(localabsoluteJitter) # make a sd F0 list
rapJitter_list.append(rapJitter)
ppq5Jitter_list.append(ppq5Jitter)
localShimmer_list.append(localShimmer)
localdbShimmer_list.append(localdbShimmer)
apq3Shimmer_list.append(apq3Shimmer)
aqpq5Shimmer_list.append(aqpq5Shimmer)
apq11Shimmer_list.append(apq11Shimmer)
hnr05_list.append(hnr05)
hnr15_list.append(hnr15)
hnr25_list.append(hnr25)
hnr35_list.append(hnr35)
hnr38_list.append(hnr38)

apred = pd.DataFrame(np.column_stack([localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, hnr05_list, hnr15_list, hnr25_list]),
                               columns=["Jitter_rel","Jitter_abs","Jitter_RAP","Jitter_PPQ","Shim_loc","Shim_dB","Shim_APQ3","Shim_APQ5","Shi_APQ11", "hnr05", "hnr15", "hnr25"])  #add these lists to pandas in the right order


In [37]:
clf.predict(apred)

array([0.])

In [9]:
import joblib
joblib.dump(clf, "trainedModel.sav")

['trainedModel.sav']